In [ ]:
from google.colab import files

uploaded = files.upload()

Saving tweets.csv to tweets.csv


In [ ]:
import gensim.downloader as api

word2vec_model = api.load("word2vec-google-news-300")

[=================================================-] 99.9% 1661.9/1662.8MB downloaded


In [ ]:
import spacy

# Download the pre-trained model
spacy.cli.download("en_core_web_md")

# Load the downloaded model
word2vec_model = spacy.load("en_core_web_md")
word2vec_model = word2vec_model.vocab.vectors

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import re
import nltk
import gensim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 1: Data Collection (assuming you have a CSV file 'tweets.csv' with 'text' and 'sentiment' columns)
import pandas as pd
data = pd.read_csv('tweets.csv')
data

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [ ]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Remove mentions
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = nltk.word_tokenize(text)  # Tokenize into words
    text = [word for word in text if word not in nltk.corpus.stopwords.words('english')]  # Remove stopwords
    return text

In [15]:
nltk.download('punkt')
nltk.download('stopwords')
data['processed_text'] = data['text'].apply(preprocess_text)

# Step 3: Word Embedding (assuming you have a pre-trained word2vec model)
import gensim.downloader as api

# List available models
print(api.info())

# Download a specific model
word2vec_model = api.load("word2vec-google-news-300")

# Step 4: Text Representation

def get_tweet_vector(tweet):
    vectors = [word2vec_model[word] for word in tweet if word in word2vec_model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(word2vec_model.vector_size)

data['tweet_vector'] = data['processed_text'].apply(get_tweet_vector)

# Step 5: Model Training
X = np.array(data['tweet_vector'].tolist())
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Step 6: Model Evaluation
pred_labels = classifier.predict(X_test)
accuracy = accuracy_score(y_test, pred_labels)
print("Accuracy:", accuracy)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1, 'record_format': 'dict', 'file_size': 6344358, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py', 'license': 'All files released for the task are free for general research use', 'fields': {'2016-train': ['...'], '2016-dev': ['...'], '2017-test': ['...'], '2016-test': ['...']}, 'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.', 'checksum': '701ea67acd82e75f95e1d8e62fb0ad29', 'file_name': 'semeval-2016-2017-task3-subtaskBC.gz',

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
